In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
# def MoG_generation(K=5):
#     data_dim = 2
#     # data_range = 6.0
#     pi_logits = np.random.sample(K)
#     pi = pi_logits / np.sum(pi_logits)

#     mu_list = [(1.0, 1.0), (1.0, 5.0), (3.0, 3.0), (5.0, 1.0), (5.0, 5.0)]
#     mu = np.array(mu_list)
    
#     adv_l = [(1.0, 3.0), (3.0, 1.0), (3.0, 5.0), (5.0, 3.0)]
#     adv_sample = np.array(adv_l)  
#     return pi, mu, adv_sample

def MoG_generation(K=5):
    samples_list = []

    while len(samples_list) < 1000:
        normal_sample = np.random.randn(data_dim)
        cls = np.random.randint(K)
        sample = normal_sample + true_mu[cls]
        if -.5 < sample[0] < 6.5 and -.5 < sample[1] < 6.5:
            samples_list.append(sample)
    samples = np.array(samples_list)
    adv_sample = [(0, 3), (3, 0), (3, 6), (6, 3)]
    

def MoG_prob(x, pi, mu, cov=None):
    K, dim = mu.shape
    if cov is None:
        cov = np.tile(np.eye(dim), (K, 1, 1))
    assert x.shape == (dim,)
    assert pi.shape == (K,)
    assert cov.shape == (K, dim, dim)
    prob = 0.0
    for k in range(K):
        s, logdet = np.linalg.slogdet(cov[k] + np.eye(dim) * 1e-3)
        # print(s, np.exp(logdet))
        assert s > 0
        log_prob_k = -dim * 0.5 * math.log(2 * math.pi) - 0.5 * logdet - 0.5 * (x - mu[k]).dot(x - mu[k])
        prob += np.exp(log_prob_k) * pi[k]
    return prob

def MoG_plot(pi, mu, cov=None):
    plt.figure(figsize=(5, 5))
    
    x1 = np.linspace(-2.0, 8.0, 101)
    x2 = np.linspace(-2.0, 8.0, 101)
    p_lists = []
    for _x2 in x2:
        p = []
        for _x1 in x1:
            p.append(MoG_prob(np.array((_x1, _x2)), pi, mu, cov) )
        p_lists.append(p)
    P = np.array(p_lists)
    
    plt.imshow(P, origin='lower', interpolation='bilinear')
    plt.xticks(np.linspace(0, 100, 6), np.linspace(-2, 8, 6))
    plt.yticks(np.linspace(0, 100, 6), np.linspace(-2, 8, 6))

def Data_plot(samples, adv_sample):
    plt.figure(figsize=(5, 5))

    x1 = [x[0] for x in samples]
    x2 = [x[1] for x in samples]
    plt.scatter(x1, x2, s=5, marker='o')
    
    x1 = [x[0] for x in adv_sample]
    x2 = [x[1] for x in adv_sample]
    plt.scatter(x1, x2, s=10, marker='X')
    
    plt.xlim(-2, 8)
    plt.ylim(-2, 8)

In [ ]:
pi, mu, adv_sample = MoG_generation()

In [ ]:
pi

In [ ]:
mu

In [ ]:
MoG_plot(np.array([0.2]*5), mu)
plt.savefig('figures/MoG-plot.png')
plt.show()

In [ ]:
samples = sample_from_MoG(pi, mu)

In [ ]:
samples[:30]

In [ ]:
adv_sample

In [ ]:
Data_plot(samples, adv_sample)
plt.savefig('Dataset.png')
plt.show()

In [ ]:
data_fname = 'data_multi_adv.npz'
load_data = np.load(data_fname)
true_pi = load_data['pi']
true_mu = load_data['mu']
samples_100 = load_data['samples']
adv_sample = load_data['adv_sample']

In [ ]:
MoG_plot(true_pi,true_mu)

In [ ]:
len(samples)

In [ ]:
K=5
data_dim=2
data_range=6.0
data_n=100
samples_list = []

while len(samples_list) < 1000:
    normal_sample = np.random.randn(data_dim)
    cls = np.random.randint(K)
    sample = normal_sample + true_mu[cls]
    if -.5 < sample[0] < 6.5 and -.5 < sample[1] < 6.5:
        samples_list.append(sample)
samples = np.array(samples_list)

In [ ]:
Data_plot(samples_100, adv_sample)
plt.savefig('figures/Dataset-100.png')
plt.show()

In [ ]:
adv_sample= [(0, 3), (3, 0), (3, 6), (6, 3)]

In [ ]:
Data_plot(samples, adv_sample)
plt.savefig('figures/Dataset-1000.png')
plt.show()

In [ ]:
data_fname = 'datasets/multi-adv-0/data_multi_adv_1000.npz'
np.savez(data_fname, pi=pi, mu=mu, samples=samples, adv_sample=adv_sample)

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=2) 

gmm.weights_ = pi
gmm.means_ = mu
gmm.covariances_ = np.array([np.eye(2)] * 5)